In [2]:
import torch
import numpy as np
from torch.utils.data import DataLoader   
from SpeechDataGenerator import SpeechDataGenerator
import torch.nn as nn
import os
import numpy as np
from torch import optim
import argparse
from models.x_vector_Indian_LID import X_vector
from sklearn.metrics import accuracy_score
from utils.utils import speech_collate
import torch.nn.functional as F
torch.multiprocessing.set_sharing_strategy('file_system')


In [3]:
class Args:
    training_filepath = 'meta/training.txt'
    testing_filepath = 'meta/testing.txt'
    validation_filepath = 'meta/validation.txt'
    input_dim = 257
    num_classes = 2
    lamda_val = 0.1
    batch_size = 1
    use_gpu = True
    num_epochs = 100
    mode = "test"
args=Args()

In [4]:
### Data related
dataloader_eval = SpeechDataGenerator(manifest=args.testing_filepath,mode='test')
dataloader_eval = DataLoader(dataloader_eval, batch_size=args.batch_size,shuffle=True,collate_fn=speech_collate) 

## Model related
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(f"device is {device}")
model = X_vector(args.input_dim, args.num_classes).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0, betas=(0.9, 0.98), eps=1e-9)
loss_fun = nn.CrossEntropyLoss()


device is cuda


In [5]:
print("start eval")
checkpoint = torch.load("save_model/best_check_point_99_0.000825372063748849")
model.load_state_dict(checkpoint['model'])
model.eval()
with torch.no_grad():
    eval_loss_list=[]
    full_preds_before=[]
    full_preds=[]
    full_gts=[]
    full_x_vec=[]
    for sample_batched in dataloader_eval:
        features = torch.from_numpy(np.asarray([torch_tensor.numpy().T for torch_tensor in sample_batched[0]])).float()
        labels = torch.from_numpy(np.asarray([torch_tensor[0].numpy() for torch_tensor in sample_batched[1]])).long()
        features, labels = features.to(device),labels.to(device)
        pred_logits,x_vec = model(features)
        full_x_vec.append(x_vec)
        #### CE loss
        loss = loss_fun(pred_logits,labels)
        eval_loss_list.append(loss.item())
        #train_acc_list.append(accuracy)
        full_preds_before.extend(pred_logits)
        prediction = np.argmax(pred_logits.detach().cpu().numpy(),axis=1)
        
        full_preds.append(np.int64(prediction[0]))
        full_gts.append(np.int64(labels.detach().cpu().numpy()[0]))
            
    mean_acc = accuracy_score(full_gts,full_preds)
    mean_loss = np.mean(np.asarray(eval_loss_list))
    print(f'Total eval loss {mean_loss} and eval accuracy {mean_acc}')
    print(full_preds_before[:2])
    print(''.join(map(str,full_preds)))
    print(''.join(map(str,full_gts)))

    model_save_path = os.path.join('save_eval', 'best_check_point_'+str(mean_loss))
    state_dict = {'model': model.state_dict(),'optimizer': optimizer.state_dict(),'epoch': checkpoint['epoch']}
    torch.save(state_dict, model_save_path)

start eval
Total eval loss 0.0021620242689630678 and eval accuracy 1.0
[tensor([-143.2951,   86.1942], device='cuda:0'), tensor([-22.6542,  13.8772], device='cuda:0')]
11111111111111110111111011101111111111111111111111111111111111111011111111111111111111111111111111111111101111101111111111111111111111111111111111111111111111111001111111101111111111111111111110111110110111101111111111111111111011111111110111111111111111101111111111111111111111111111111111111
11111111111111110111111011101111111111111111111111111111111111111011111111111111111111111111111111111111101111101111111111111111111111111111111111111111111111111001111111101111111111111111111110111110110111101111111111111111111011111111110111111111111111101111111111111111111111111111111111111


In [6]:
for idx, x_vec in enumerate(full_x_vec):
    key_len = 64
    b = F.normalize(x_vec,dim=0)
    b = torch.split(x_vec,[512//key_len]*(key_len),dim=1)
    b = torch.stack(list(b), dim=1)
    b = torch.sum(b,dim=2)
    generated = ""
    for i in b[0]:
        if(i >= 0):
            generated += "1"
        else :
            generated += "0"
    if(full_gts[idx] == 1):
        print(f"label = {full_gts[idx]}, pred = {full_preds[idx]} ")
        print(generated, hex(int(generated, 2)))
        if(idx == 10):
            break
    # if(full_gts[idx] == 0):
    #     print(f"label = {full_gts[idx]}, pred = {full_preds[idx]} ")
    #     print(generated, hex(int(generated, 2)))
    #     if(idx == 10):
    #         break

label = 1, pred = 1 
1000111100011000101010100100001001100010011111101111001101011000 0x8f18aa42627ef358
label = 1, pred = 1 
1000111100011000101011100100001001100010011111101111001101011000 0x8f18ae42627ef358
label = 1, pred = 1 
1000111100011000101010100100001001100010011111101111001101011000 0x8f18aa42627ef358
label = 1, pred = 1 
1000111100011000101010100100001001100010011111101111001101011000 0x8f18aa42627ef358
label = 1, pred = 1 
1000111100011000101010100100001001100010011111101111001101011000 0x8f18aa42627ef358
label = 1, pred = 1 
1000111100011000101011100100001001100000001111101100000101011000 0x8f18ae42603ec158
label = 1, pred = 1 
1000111100011000101011100100001001100010011111101111001101011000 0x8f18ae42627ef358
label = 1, pred = 1 
1000111100011000101011100100001001100010011111101111001101011000 0x8f18ae42627ef358
label = 1, pred = 1 
1000111100011000101010100100001001100010011111101111001101011000 0x8f18aa42627ef358
label = 1, pred = 1 
1000111100011000101011100100001001

In [21]:
len(full_x_vec)

293

In [22]:
for idx, x_vec in enumerate(full_x_vec):
    key_len = 512
    b = F.normalize(x_vec,dim=0)
    b = torch.split(x_vec,[512//key_len]*(key_len),dim=1)
    b = torch.stack(list(b), dim=1)
    b = torch.sum(b,dim=2)
    generated = ""
    for i in b[0]:
        if(i >= 0):
            generated += "1"
        else :
            generated += "0"
    if(full_gts[idx] == 1):
        print(f"label = {full_gts[idx]}, pred = {full_preds[idx]} ")
        print(generated, hex(int(generated, 2)))
        if(idx == 20):
            break
    if(full_gts[idx] == 0):
        print(f"label = {full_gts[idx]}, pred = {full_preds[idx]} ")
        print(generated, hex(int(generated, 2)))
        if(idx == 20):
            break

label = 1, pred = 1 
11010100000101010011110000010101111100110100011110010111001111110000001101000111111000001110110011100010100110110011001100110001001111011000001011101001110011100111000111000100011110010111100100010100101101111001001001100100110110101100001011101111010110010000000111010011111111011001011110011001001100001101010100010101000110011001110001111011101110011110010111001101100001110100000000111111111011011010101000101011100000000000001010011111010111110000001001110010000100010001111101101101101110000001001011000000 0xd4153c15f347973f0347e0ece29b33313d82e9ce71c4797914b79264dac2ef5901d3fd979930d515199c7bb9e5cd87403fedaa2b80029f5f0272111f6db812c0
label = 1, pred = 1 
11010100000101010011110000010101111100110100011110010111001111110000001101000111111000001110110011100010100110110011001100110001001111011000001011101001110011100111000111000100011110010111100100010100101101111101001001100100110110101100001011101111010110011000000111010011111111011001011110011001001100001101010100

In [8]:
print(x_vec.shape,x_vec[0][:20])


torch.Size([1, 512]) tensor([ 10.6081,   5.4709,  -5.3112,   1.7491,  -5.0190,  13.8130,  -6.7945,
         -5.0039,  -6.6947,  -8.4571,  -0.6471,   3.5376, -18.7250,   3.5002,
        -20.2084,   0.2550,  -4.2026,  -2.7005,   1.3593,   3.9153],
       device='cuda:0')


In [16]:
b = F.normalize(x_vec,1)
print(b.shape,b[0])

torch.Size([1, 512]) tensor([ 3.0527e-03,  1.5744e-03, -1.5284e-03,  5.0334e-04, -1.4443e-03,
         3.9749e-03, -1.9552e-03, -1.4400e-03, -1.9265e-03, -2.4337e-03,
        -1.8621e-04,  1.0180e-03, -5.3885e-03,  1.0072e-03, -5.8153e-03,
         7.3372e-05, -1.2094e-03, -7.7712e-04,  3.9118e-04,  1.1267e-03,
         8.9656e-04,  1.9139e-03, -1.1508e-03, -2.2102e-03, -5.9780e-03,
        -2.0752e-03, -1.0307e-02,  1.1581e-04, -1.1183e-03,  1.6360e-03,
        -2.7871e-03,  1.6860e-03,  6.2826e-04,  1.8025e-03,  8.4242e-03,
         2.6100e-03, -1.5885e-03,  1.1110e-04,  1.6182e-03,  2.8339e-04,
        -1.0819e-03,  4.5757e-03, -1.1439e-04, -1.1642e-03, -1.1857e-03,
         9.6979e-04,  4.2271e-03,  1.2791e-03,  2.5213e-03, -1.8458e-03,
         2.4671e-04,  2.7500e-03,  2.7312e-04,  1.1805e-03,  1.3490e-03,
         2.9652e-03, -1.7172e-03, -4.6980e-04,  1.7195e-03,  1.5348e-03,
         1.1551e-03,  1.3045e-03,  1.0843e-03,  2.3904e-03, -1.7249e-03,
        -2.5668e-03, -1.5365e-

In [12]:
print(x_vec[0])

tensor([ 1.0608e+01,  5.4709e+00, -5.3112e+00,  1.7491e+00, -5.0190e+00,
         1.3813e+01, -6.7945e+00, -5.0039e+00, -6.6947e+00, -8.4571e+00,
        -6.4709e-01,  3.5376e+00, -1.8725e+01,  3.5002e+00, -2.0208e+01,
         2.5497e-01, -4.2026e+00, -2.7005e+00,  1.3593e+00,  3.9153e+00,
         3.1156e+00,  6.6508e+00, -3.9992e+00, -7.6805e+00, -2.0774e+01,
        -7.2114e+00, -3.5818e+01,  4.0243e-01, -3.8860e+00,  5.6853e+00,
        -9.6853e+00,  5.8589e+00,  2.1832e+00,  6.2637e+00,  2.9274e+01,
         9.0699e+00, -5.5202e+00,  3.8608e-01,  5.6232e+00,  9.8478e-01,
        -3.7596e+00,  1.5901e+01, -3.9749e-01, -4.0455e+00, -4.1202e+00,
         3.3700e+00,  1.4689e+01,  4.4449e+00,  8.7615e+00, -6.4142e+00,
         8.5733e-01,  9.5562e+00,  9.4910e-01,  4.1021e+00,  4.6877e+00,
         1.0304e+01, -5.9673e+00, -1.6326e+00,  5.9752e+00,  5.3334e+00,
         4.0138e+00,  4.5330e+00,  3.7680e+00,  8.3068e+00, -5.9942e+00,
        -8.9197e+00, -5.3395e+00, -1.5084e+01, -2.7

In [10]:
b.shape

torch.Size([1, 512])

In [142]:
b = F.normalize(x_vec,dim=0)
print(b.shape,b[0][:20])


torch.Size([1, 512]) tensor([ 1.,  1., -1.,  1., -1.,  1., -1., -1., -1., -1., -1.,  1., -1.,  1.,
        -1.,  1., -1., -1.,  1.,  1.], device='cuda:0')
